In [1]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

# Build paths
# source_path = Path(DOWNLOAD_DIR) / f'df_finviz_{date_str}.pkl'
# dest_path = Path(DEST_DIR) / 'df_finviz.pkl'
source_path = Path(DOWNLOAD_DIR) / f'df_finviz_{date_str}.parquet'
dest_path = Path(DEST_DIR) / 'df_finviz.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

# Example file operations:
# import pandas as pd
# df = pd.read_pickle(source_path)
# df.to_pickle(dest_path)

source_path: C:\Users\ping\Downloads\df_finviz_2025-04-02.parquet
dest_path: ..\data\df_finviz.parquet


In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell

In [3]:
df = pd.read_parquet(source_path, engine='pyarrow')
# df = pd.read_pickle(source_path)
display(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   No.              1560 non-null   object
 1   Ticker           1560 non-null   object
 2   Company          1560 non-null   object
 3   Sector           1560 non-null   object
 4   Industry         1560 non-null   object
 5   Market Cap       1560 non-null   object
 6   Dividend         1560 non-null   object
 7   Perf Week        1560 non-null   object
 8   Perf Month       1560 non-null   object
 9   Perf Quart       1560 non-null   object
 10  Perf Half        1560 non-null   object
 11  Perf Year        1560 non-null   object
 12  Perf YTD         1560 non-null   object
 13  Beta             1560 non-null   object
 14  ATR              1560 non-null   object
 15  Volatility W     1560 non-null   object
 16  Volatility M     1560 non-null   object
 17  SMA20            1560 non-null   

None

,No.,Ticker,Company,Sector,Industry,Market Cap,Dividend,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,All-Time High,All-Time Low,RSI,Gap,Avg Volume,Rel Volume,Volume,Price,Change,Single Category,Asset Type,AUM
0,181,WMB,Williams Cos Inc,Energy,Oil & Gas Midstream,75.19B,3.25%,2.75%,8.45%,13.74%,34.94%,58.48%,13.82%,0.68,1.49,2.16%,2.52%,5.69%,7.32%,20.31%,-0.11%,16.69%,-0.11%,65.67%,-0.11%,9587.73%,66.05,-1.09%,7.70M,0.90,"6,926,568",61.60,1.70%,-,-,-
1,182,SCCO,Southern Copper Corporation,Basic Materials,Copper,75.13B,3.65%,-3.42%,9.82%,3.94%,-17.32%,-9.22%,4.30%,1.04,3.34,3.83%,3.38%,-0.95%,0.41%,-6.56%,-8.98%,11.89%,-25.90%,11.89%,-25.90%,6919.18%,48.00,-1.02%,1.39M,0.64,"892,396",94.36,0.51%,-,-,-
2,183,RIO,Rio Tinto plc ADR,Basic Materials,Other Industrial Metals & Mining,75.10B,6.34%,-4.72%,-3.68%,2.24%,-15.84%,-6.67%,1.85%,0.72,1.21,1.49%,1.56%,-3.63%,-3.37%,-5.49%,-7.47%,2.10%,-19.32%,3.54%,-44.25%,961.10%,38.37,-0.40%,3.18M,0.73,"2,330,586",59.90,-0.55%,-,-,-
3,184,GD,General Dynamics Corp,Industrials,Aerospace & Defense,74.63B,2.14%,2.40%,10.06%,4.81%,-8.66%,-5.32%,4.76%,0.50,4.87,1.18%,2.10%,3.20%,6.33%,-2.00%,-1.15%,15.08%,-12.89%,15.08%,-12.89%,19446.63%,67.78,-0.88%,1.69M,0.80,"1,348,564",276.04,0.69%,-,-,-
4,185,NOC,Northrop Grumman Corp,Industrials,Aerospace & Defense,74.14B,1.68%,0.99%,10.42%,9.79%,-3.01%,8.66%,9.14%,0.14,10.98,1.61%,2.52%,3.20%,6.63%,5.06%,-1.29%,20.16%,-7.81%,22.36%,-7.93%,8145.59%,65.97,-0.49%,1.01M,0.74,"749,613",512.18,-0.18%,-,-,-


In [4]:
import numpy as np

def convert_B_M_K_to_million(value_str):
  """
  Convert financial string values with suffixes to numeric values in billions
  Examples:
  '104.27B' -> 104,270
  '104.27M' -> 104.27
  '104.27K' -> 0.10427
  '-' -> np.nan
  """
  
  if not isinstance(value_str, str):
    return value_str  # Return the original value if not a string
    
  if value_str == '-':
    return np.nan
    
  # Remove any whitespace
  value_str = value_str.strip()
  
  # Handle suffixes
  multipliers = {
    'B': 1000,  
    'M': 1,
    'K': 0.001
  }
  
  try:
    # Extract the numeric part and suffix
    if value_str[-1] in multipliers:
      number = float(value_str[:-1])
      multiplier = multipliers[value_str[-1]]
      return number * multiplier
    else:
      # If no suffix, try to convert directly to float
      return float(value_str)
  except (ValueError, IndexError):
    return np.nan

# Example usage:
# df['Market Cap Numeric'] = df['Market Cap'].apply(convert_B_M_K_to_million)

In [5]:
import pandas as pd
import numpy as np

def process_percentage_columns(df):
    """
    Identifies and processes columns in a DataFrame where values end with '%'.
    The function cleans, converts to numeric, renames, and prints which columns were modified.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The modified DataFrame with processed percentage columns.
                      Returns the original DataFrame if no percentage columns are found.
    """

    # Identify columns where values END WITH '%'
    percent_cols = [
        col for col in df.columns
        if df[col].dtype == 'object'
        and df[col].str.strip().str.endswith('%', na=False).any()
    ]

    # If no percentage columns are found, return the original DataFrame
    if not percent_cols:
        print("No percentage columns found to modify.")  # Print message
        return df

    print("The following columns ending with % were modified:") #Print message before the loop

    # Process identified percentage columns
    for col in percent_cols:
        # Clean data: (1) Strip whitespace, (2) Handle '-', (3) Remove trailing %
        cleaned_series = (
            df[col].str.strip()
            .replace('-', np.nan)  # Convert '-' to NaN
            .str.replace(r'%$', '', regex=True)  # Remove only ENDING %
        )
        
        # Convert to numeric (coerce invalid values to NaN)
        df[col] = pd.to_numeric(cleaned_series, errors='coerce')
        
        # Rename column
        df.rename(columns={col: f"{col} %"}, inplace=True)
        print(f"- {col}")  # Print the column name
        
    return df

# Example Usage:
# Assuming you have a DataFrame called 'df'
# df = process_percentage_columns(df)

In [6]:
df = process_percentage_columns(df)

The following columns ending with % were modified:
- Dividend
- Perf Week
- Perf Month
- Perf Quart
- Perf Half
- Perf Year
- Perf YTD
- Volatility W
- Volatility M
- SMA20
- SMA50
- SMA200
- 50D High
- 50D Low
- 52W High
- 52W Low
- All-Time High
- All-Time Low
- Gap
- Change


In [7]:
# List of columns to concatenate
columns_to_concat = ["Sector", "Industry", "Single Category", "Asset Type"]

# Replace '-' with empty string in specified columns
for col in columns_to_concat:
    df[col] = df[col].replace('-', '')

# Concatenate the columns, handling empty strings, and remove extra spaces
df['Info'] = df[columns_to_concat].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)

In [8]:
# Concatenate the columns 'Market Cap' and 'AUM'
# Replace '-' with empty string in both columns

df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')

In [9]:
# Create new column by converting to numeric values in millions
df['MktCap AUM, M'] = df['MktCap AUM'].apply(convert_B_M_K_to_million)
df['Avg Volume, M'] = df['Avg Volume'].apply(convert_B_M_K_to_million)
df['Volume, M'] = (pd.to_numeric(df['Volume'].str.replace(',', ''), errors='coerce') / 1_000_000)

In [10]:
# Convert 'Beta', 'ATR', 'Rel Volume', and 'Price' to float
for col in ['Beta', 'ATR', 'RSI', 'Rel Volume', 'Price']:
  # Clean and convert to numeric
  df[col] = pd.to_numeric(
    df[col].str.replace('$', '').str.replace(',', ''),
    errors='coerce'  # Convert invalid values to NaN
  )

# Verify the conversion
for col in ['Beta', 'ATR', 'Rel Volume', 'Price']:
  print(f"{col}: {df[col].dtype}")

Beta: float64
ATR: float64
Rel Volume: float64
Price: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   No.              1560 non-null   object 
 1   Ticker           1560 non-null   object 
 2   Company          1560 non-null   object 
 3   Sector           1560 non-null   object 
 4   Industry         1560 non-null   object 
 5   Market Cap       1560 non-null   object 
 6   Dividend %       1223 non-null   float64
 7   Perf Week %      1559 non-null   float64
 8   Perf Month %     1559 non-null   float64
 9   Perf Quart %     1555 non-null   float64
 10  Perf Half %      1553 non-null   float64
 11  Perf Year %      1544 non-null   float64
 12  Perf YTD %       1560 non-null   float64
 13  Beta             1544 non-null   float64
 14  ATR              1560 non-null   float64
 15  Volatility W %   1559 non-null   float64
 16  Volatility M %   1559 non-null   float64
 17  SMA20 %       

In [12]:
df.columns

Index(['No.', 'Ticker', 'Company', 'Sector', 'Industry', 'Market Cap',
       'Dividend %', 'Perf Week %', 'Perf Month %', 'Perf Quart %',
       'Perf Half %', 'Perf Year %', 'Perf YTD %', 'Beta', 'ATR',
       'Volatility W %', 'Volatility M %', 'SMA20 %', 'SMA50 %', 'SMA200 %',
       '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %',
       'All-Time Low %', 'RSI', 'Gap %', 'Avg Volume', 'Rel Volume', 'Volume',
       'Price', 'Change %', 'Single Category', 'Asset Type', 'AUM', 'Info',
       'MktCap AUM', 'MktCap AUM, M', 'Avg Volume, M', 'Volume, M'],
      dtype='object')

In [13]:
my_cols = ['Ticker', 'Company', 'Info', 'MktCap AUM, M', 'Beta',
          'RSI', 'Perf YTD %', 'Perf Week %', 'Perf Month %', 'Perf Quart %', 'Perf Half %', 'Perf Year %',
          'SMA20 %', 'SMA50 %', 'SMA200 %',   
          '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %', 'All-Time Low %',
          'ATR', 'Volatility W %', 'Volatility M %',  
          'Volume, M', 'Avg Volume, M','Rel Volume',
          'Price', 'Change %', 'Dividend %', 
          ]

In [14]:
# Create new df with my_cols and set Ticker as index
new_df = df[my_cols].set_index('Ticker')

# Sort by 'MktCap AUM, M' in descending order, with NaN values last
new_df = new_df.sort_values(by='MktCap AUM, M', ascending=False, na_position='last')

# Display info and first few rows to verify
display(new_df.info(), new_df)

<class 'pandas.core.frame.DataFrame'>
Index: 1560 entries, AAPL to FNDC
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1560 non-null   object 
 1   Info             1560 non-null   object 
 2   MktCap AUM, M    1560 non-null   float64
 3   Beta             1544 non-null   float64
 4   RSI              1559 non-null   float64
 5   Perf YTD %       1560 non-null   float64
 6   Perf Week %      1559 non-null   float64
 7   Perf Month %     1559 non-null   float64
 8   Perf Quart %     1555 non-null   float64
 9   Perf Half %      1553 non-null   float64
 10  Perf Year %      1544 non-null   float64
 11  SMA20 %          1560 non-null   float64
 12  SMA50 %          1560 non-null   float64
 13  SMA200 %         1560 non-null   float64
 14  50D High %       1560 non-null   float64
 15  50D Low %        1560 non-null   float64
 16  52W High %       1560 non-null   float64
 17  52W Low %       

None

,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,All-Time High %,All-Time Low %,ATR,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Dividend %
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3363290.0,1.27,48.97,-10.59,1.07,-5.10,-11.23,-3.91,31.68,1.44,-2.72,-2.31,-10.44,7.42,-13.92,36.46,-13.92,351839.72,5.90,2.68,2.76,32.288560,52.69000,0.61,223.89,0.31,0.46
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2840820.0,0.99,42.23,-9.34,-2.01,-1.66,-10.05,-11.19,-9.99,-1.09,-5.18,-9.37,-14.77,4.06,-18.41,4.06,-18.41,479501.58,8.75,2.30,2.19,15.714081,23.06000,0.68,382.14,-0.01,0.86
NVDA,NVIDIA Corp,"Technology, Semiconductors",2694250.0,2.11,41.01,-17.77,-2.94,-4.80,-19.69,-9.07,22.20,-3.42,-10.42,-13.33,-25.88,6.53,-27.89,46.05,-27.89,331159.98,5.56,4.58,4.28,204.695085,280.50000,0.73,110.42,0.25,0.04
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",2077260.0,1.31,43.14,-10.66,-2.55,-3.82,-11.43,5.20,8.31,-0.49,-8.72,-2.15,-19.18,6.30,-19.18,29.29,-19.18,298581.91,6.79,3.30,3.08,50.483318,40.07000,1.26,196.01,2.00,NaN
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1924390.0,1.02,37.15,-16.58,-4.95,-7.97,-17.56,-4.98,1.51,-4.18,-11.41,-9.86,-23.88,4.37,-23.88,7.19,-23.88,553.44,4.93,3.41,2.87,16.492587,19.97000,0.83,158.86,-0.01,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BKLC,BNY Mellon US Large Cap Core Equity ETF,"Financial, Exchange Traded Fund, US Equities -...",3190.0,1.03,43.95,-3.87,-1.15,-2.37,-4.27,-1.68,8.44,-0.10,-3.98,-1.77,-8.32,2.98,-8.32,14.71,-8.32,109.21,1.81,1.68,1.60,0.240570,0.11205,2.15,107.72,0.65,1.27
SPGP,Invesco S&P 500 GARP ETF,"Financial, Exchange Traded Fund, US Equities -...",3190.0,1.01,47.83,-3.48,-0.68,0.33,-3.09,-3.29,-5.14,0.93,-3.23,-3.54,-9.15,5.05,-10.33,6.54,-10.33,490.66,1.80,1.88,1.75,0.107510,0.26187,0.41,101.23,0.95,1.51
IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,"Financial, Exchange Traded Fund, Bonds - Corpo...",3170.0,0.10,38.30,0.12,-0.28,0.00,0.12,-0.08,1.46,-0.16,-0.15,0.21,-0.32,0.04,-0.32,1.62,-10.37,11.54,0.02,0.05,0.06,0.271928,0.55429,0.49,25.08,-0.04,3.86


In [15]:
# Save to pickle file
# new_df.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
new_df.to_parquet(dest_path, engine='pyarrow')
print(f'save new_df to {dest_path}')

# # To load it later:
# loaded_df = pd.read_pickle('df_finviz.pkl')

save new_df to ..\data\df_finviz.parquet
